[16 PyTorch 神经网络基础【动手学深度学习v2】](https://www.bilibili.com/video/BV1AK4y1P7vs?spm_id_from=333.999.0.0)

### 搭积木-层与块的构建

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
# 简单的线性层可以如下构建
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
x = torch.rand(2,20)
x,net(x)

(tensor([[0.8829, 0.7536, 0.2120, 0.6006, 0.0111, 0.9008, 0.6258, 0.9156, 0.7527,
          0.0798, 0.4771, 0.3603, 0.0095, 0.2410, 0.7011, 0.1737, 0.7260, 0.8787,
          0.9079, 0.0936],
         [0.0974, 0.7450, 0.5547, 0.9807, 0.7628, 0.3972, 0.7241, 0.5447, 0.9490,
          0.9260, 0.6456, 0.6435, 0.5539, 0.4542, 0.0190, 0.2692, 0.9041, 0.2310,
          0.4011, 0.5087]]),
 tensor([[-0.1069,  0.0681, -0.1217,  0.2846,  0.2035, -0.0414, -0.1511,  0.0792,
          -0.0577,  0.0839],
         [-0.0409, -0.0594, -0.0635,  0.2406,  0.1542, -0.0577, -0.0530,  0.0860,
           0.0064, -0.0692]], grad_fn=<AddmmBackward0>))

In [3]:
# 自定义模块
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256) # 隐藏层
        self.out = nn.Linear(256,10) # 输出层
    
    # 注意，这⾥我们使⽤ReLU的函数版本，其在nn.functional模块中定义。
    # 为了能够相应__call__函数,这里必须是forward函数
    def forward(self,x): 
        return self.out(F.relu((self.hidden(x))))

In [4]:
# 这种用法是直接调用了 nn.Module.__call__函数,详情见__call__用法
nn.Module.__call__
net = MLP()
net(x)

tensor([[-0.3236, -0.1438, -0.0954, -0.0510,  0.2201, -0.0953,  0.0853, -0.1131,
         -0.0542,  0.1266],
        [-0.1849, -0.0926, -0.1060,  0.0335,  0.2042, -0.0432,  0.1339, -0.1008,
         -0.1326,  0.1557]], grad_fn=<AddmmBackward0>)

In [5]:
# __call__函数用法
class CLanguage:
    # 定义__call__方法
    def __call__(self,name,add):
        print("调用__call__()方法",name,add)
clangs = CLanguage()
clangs("C语言中文网","http://c.biancheng.net")

调用__call__()方法 C语言中文网 http://c.biancheng.net


In [6]:
# 构建顺序块
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            # pytorch会自动将_module识别为一个字典
            # self._modules: Dict[str, Optional['Module']] = OrderedDict()
            self._modules[block] = block
    
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(x)

tensor([[ 0.0995,  0.1899, -0.1976,  0.0917, -0.0798, -0.1913,  0.0147, -0.0160,
          0.3183, -0.2613],
        [ 0.2019,  0.0615, -0.1142,  0.1981,  0.0121, -0.2362,  0.0477, -0.0863,
          0.0961, -0.3274]], grad_fn=<AddmmBackward0>)

In [7]:
net._modules

OrderedDict([(Linear(in_features=20, out_features=256, bias=True),
              Linear(in_features=20, out_features=256, bias=True)),
             (ReLU(), ReLU()),
             (Linear(in_features=256, out_features=10, bias=True),
              Linear(in_features=256, out_features=10, bias=True))])

In [8]:
# 当我们不满足于使用系统框架的时候,可以利用pytorch做更多的事情
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)
    
    def forward(self,x):
        x = self.linear(x)
        # 使⽤创建的常量参数以及`relu`和`dot`函数。
        x = torch.mm(x,self.rand_weight)+1
        x = F.relu(x)
        # 复⽤全连接层。这相当于两个全连接层共享参数。
        x = self.linear(x)
        # 控制流
        while x.abs().sum()>1:
            x /=2
        return x.sum()

net = FixedHiddenMLP()
net(x)

tensor(-0.0248, grad_fn=<SumBackward0>)

In [9]:
# 混合模型,各种套法都可以试一试
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__() 
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU()) 
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP()) 
chimera(x)

tensor(0.3545, grad_fn=<SumBackward0>)

### 参数管理
- 我们首先关注具有单隐藏层的多层感知机

In [10]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
x = torch.rand(size=(2,4))
net(x)

tensor([[0.0493],
        [0.0895]], grad_fn=<AddmmBackward0>)

In [11]:
# 参数访问
# 访问的是nn.Sequential里面第三个,也就是nn.Linear(8,1)的参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0096, -0.1305,  0.0609,  0.1123,  0.1792,  0.0620,  0.1073, -0.2436]])), ('bias', tensor([0.0002]))])


In [12]:
# 目标参数获取
print(type(net[2].bias)) 
print(net[2].bias) 
print(net[2].bias.data)
print(net[2].weight) 

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0002], requires_grad=True)
tensor([0.0002])
Parameter containing:
tensor([[-0.0096, -0.1305,  0.0609,  0.1123,  0.1792,  0.0620,  0.1073, -0.2436]],
       requires_grad=True)


In [13]:
# 现在没有做反向传播,没有梯度
net[2].weight.grad

In [14]:
# 一次性访问第一个全连接层的所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()]) 
# 第二层为RElU激活函数,没有梯度
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [15]:
# data直接获取参数
net.state_dict()['2.bias'].data

tensor([0.0002])

In [16]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential() 
    for i in range(4):
    # 在这⾥嵌套
    # 
        net.add_module(f'block {i}', block1()) 
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1)) 
rgnet(x)

tensor([[0.0715],
        [0.0714]], grad_fn=<AddmmBackward0>)

### 补充知识
[torch.nn.Module.add_module(name, module)](https://blog.csdn.net/m0_46653437/article/details/112649366)
- 将一个子模块添加到当前模块中.
- 通过给定的名字,我们就可以以访问属性的方式来访问该模块.

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        # nn.init,直接给指定tensor进行赋值
        nn.init.normal_(m.weight,mean = 0 , std = 0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0022,  0.0042, -0.0097, -0.0064]), tensor(0.))

In [19]:
# 同样是赋值，不过是常数
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [20]:
def xavier(m):
    if type(m) == nn.Linear:    
        # xavier初始化
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
# net的某个部分应用对应的初始化方式
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1791,  0.1279, -0.6880, -0.1437])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [21]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        # 此次*用法类似*args
        print("Init", *[(name, param.shape)for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 只要大5的部分
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -7.1643,  0.0000,  9.6494],
        [-0.0000, -0.0000, -0.0000, -8.2174]], grad_fn=<SliceBackward0>)

In [22]:
# 也可以简单粗暴直接怼
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -6.1643,  1.0000, 10.6494])

In [23]:
# 参数绑定
# 我们需要给共享层⼀个名称，以便可以引⽤它的参数。
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同⼀个对象，⽽不只是有相同的值。
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 读写文件

In [24]:
# 张量保存
x = torch.arange(4)
torch.save(x,'x-file')
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [25]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [26]:
# 我们甚⾄可以写⼊或读取从字符串映射到张量的字典。当我们要读取或写⼊模型中的所有权重时，这很⽅便。
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

# 加载和保存模型参数

In [27]:
### 关键
# 只保存参数而不是整个模型
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

接下来，我们将模型的参数存储为⼀个叫做“mlp.params”的⽂件。

In [28]:
torch.save(net.state_dict(), 'mlp.params')

- 模型加载
- 为了恢复模型，我们实例化了原始多层感知机模型的⼀个备份。我们没有随机初始化模型参数，而是直接读
取⽂件中存储的参数。

In [29]:

clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [30]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])